## Cost Function and Backpropagation

- **Cost Function**

    - Let's first define a few variables that we will need to use:

        - L = total number of layers in the network

        - $s_l$ = number of units (not counting bias unit) in layer l

        - K = number of output units/classes
    - For neural networks, the cost function is going to be slightly more complicated:
    
    $$J(\Theta) = -\dfrac{1}{m}\sum_{i=1}^m\sum_{k=1}^K[y_k^{(i)}log((h_\Theta(x^{(i)})) + (1-y_k^{(i)})log(1-h_\Theta(x^{(i)}))_k)] + \dfrac{\lambda}{2m}\sum_{l=1}^{L-1}\sum_{i=1}^{s_l}\sum_{j=1}^{s_{l+1}}(\Theta_{j,i}^{(l)})^2$$

- **Backpropagation Algorithm**

    - "Backpropagation" is neural-network terminology for minizing our cost function, just like what we were doing with gradient descent in logistic and linear regression. Our goal is to compute: $min_\Theta J(\Theta)$

    - That is, we want to minimize our cost function J using an optimal set of parameters in theta. In this section, we'll look at the equations we use to compute the partial derivative of $J(\Theta)$: $\dfrac{\partial}{\partial\Theta_{i,j}^{(l)}}J(\Theta)$

    - Algorithm:

        **Backpropagation algorithm**

        - Given training set {$(x^{(1)},y^{(1)}),...,(x^{(m)},y^{(m)})$}

        - Set $\Delta_{i,j} := 0$ for all (l,i,j)

        - For training example t = 1 to m:

        1. Set $a^{(1)} := x^{(t)}$

        2. Perform forward propagation to compute $a^{(l)}$ for l=2, 3, ..., L

        3. Using $y^{(t)}$ compute $\delta^{(L)} = a^{(L)} - y^{(t)}$

        Where L is our total number of layers and $a^{(L)}$ is the vector of outputs of the activation units for the last layer. So our "error values" for the last layer are simply the differences of our actual results in the last layer and the correct outputs in y. To get the delta values of the layers before the last layer, we can use an equation that steps us back from right to left:

        4. Compute $\delta^{(L-1)}, \delta^{(L-2)}, ..., \delta^{(2)}$ using $\delta^{(l)} = ((\Theta^{(l)})^T\delta^{(l+1)}).*a^{(l)}.*(1-a^{(l)})$

        The delta values of layer l are calculated by multiplying the delta values in the next layer with the theta matrix of layer l. We then element-wise multiply that with a function called g', or g-prime, which is the derivative of the activation function g evaluated with the input values given by $z^{(l)}$.

        The g-prime derivative terms can also be writtten out as:

        $g'(z^{(l)}) = a^{(l)}.*(1-a^{(l)})$

        5. $\Delta_{i,j}^{(l)} := \Delta_{i,j}^{(l)} + a_j^{(l)}\delta_j^{(l+1)}$ or with vectorization, $\Delta^{(l)} := \Delta^{(l)} + \delta^{(l+1)}(a^{(l)})^T$

        Hence we update our new $\Delta$ matrix

        - $D_{i,j} := \dfrac{1}{m}(\Delta_{i,j}^{(l)} + \lambda\Theta_{i,j}^{(l)})$, if j!=0

        - $D_{i,j} := \dfrac{1}{m}(\Delta_{i,j}^{(l)}$, if j=0

        The capital-delta matrix D is used as an "accumulator" to add up our values as we go along and eventually compute our partial derivative. Thus we get $\frac{\partial}{\partial\Theta_{i,j}^{(l)}} = D_{i,j}^{(l)}$